In [1]:
from obspy               import *
from obspy.clients        import fdsn
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors   import Normalize
from obspy.geodetics.base       import gps2dist_azimuth
from obspy.geodetics import locations2degrees
from obspy.clients.fdsn    import Client, RoutingClient
from obspy.core.util.obspy_types import CustomComplex
from obspy.signal.rotate import rotate2zne
from pandas import date_range
from datetime import datetime

import os
import sys
import obspy.signal.array_analysis as AA
import matplotlib.pyplot           as plt
import matplotlib.cm               as cm
import numpy                       as np
import scipy                       as sp
import scipy.odr                   as odr
import math
import obspy
import timeit

import warnings
warnings.filterwarnings('ignore')

In [3]:
## generate configuration object
config = {}

config['fdsn_client'] = Client(base_url="http://jane", timeout=200)
config['fdsn_client2'] = Client("LMU")

config['year'] = 2019
config['name'] = "all"

#starttime for data process
config['starttime'] = UTCDateTime(f"{config['year']}-01-01T00:00")
config['days'] = 59
config['dates'] = date_range(start=datetime.strptime(str(config['starttime']), '%Y-%m-%dT%H:%M:%S.%fZ'), end=datetime.strptime(str(config['starttime']+config['days']*86400), '%Y-%m-%dT%H:%M:%S.%fZ'))

# config['save_path'] = "/home/andbro/kilauea-data/LNM/data/GRF/"
config['save_path'] = f"/export/data/LNM/data/RMY/{config['name']}/"

#PFO array information
config['network'] = 'BW'
config['array_stations'] = ['FUR','FFB1','FFB2','FFB3','GELB','GRMB','TON','ALFT','BIB']

## define subarrays
config['subarray_mask'] = [0,1,2,3,4,5,6,7,8] # all
#config['subarray_mask'] = [0,1,2,3] # only inner


config['misorientations'] =  np.arange(0, len(config['array_stations']))
# config['array_station'] =[]
# config['misorientation'] =[]

# config['subarray'] = np.arange(len(config['array_stations']))

config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray_mask']]
config['subarray_misorientation'] = [config['misorientations'][i] for i in config['subarray_mask']]

config['samples'] = 86400*20

#parameter for array-derivation
# config['prefilt'] = (0.001, 0.01, 5, 10)
config['freq1'] = 0.014   #0.014 for Spudich    and  0.073 for Langston
config['freq2'] = 1.5
config['bpf'] = True
# config['channel_resp'] = 'BHZ'
config['debug'] = True

# adr parameters -> not required for rotations only (but for strain)
config['vp'] = 6264. #1700
config['vs'] = 3751. #1000
config['sigmau'] = 1e-7

In [13]:


inven = config['fdsn_client'].get_stations(network="BW",
                                           station="FFB1",
                                           channel='BHZ',
                                           starttime=config['starttime'],
                                           endtime=config['starttime']+86400,
                                           level='response'
                                          )

In [16]:
def __get_inventory_and_distances(config):
    coo = []
    for i, station in enumerate(config['subarray_stations']):

        if station == "FUR":
            net = "GR"
        else:
            net = config['network']
        try:
            inven = config['fdsn_client'].get_stations(network=net,
                                                       station=station,
                                                       channel='BHZ',
                                                       starttime=config['starttime'],
                                                       endtime=config['starttime']+86400,
                                                       level='response'
                                                      )
        except:
            print(net, station)
            continue 
            
        l_lon =  float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['longitude'])
        l_lat =  float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['latitude'])
        height = float(inven.get_coordinates('%s.%s..BHZ'%(net,station))['elevation'])
        if i == 0:
            o_lon, o_lat, o_height = l_lon, l_lat, height

        lon, lat = obspy.signal.util.util_geo_km(o_lon,o_lat,l_lon,l_lat)
        coo.append([lon*1000,lat*1000,height-o_height])  #convert unit from km to m

    return np.array(coo)

In [5]:
def __check_samples_in_stream(st, config):

    for tr in st:
        if tr.stats.npts != config['samples']:
            print(f" -> removing {tr.stats.station} due to improper number of samples ({tr.stats.npts} not {config['samples']})")
            st.remove(tr)

    return st

In [6]:
def __get_data(config):


    tbeg=config['starttime'];
    tend=tbeg+86400

    config['subarray'] = []
    tsz, tsn, tse = [],[],[]
    for k, station in enumerate(config['array_stations']):

        if k in config['subarray_mask']:

            if station == "FUR":
                net = "GR"
            else:
                net = config['network']

            print(' requesting '+net+'.'+station+'..BH*' if config['debug'] ==True else None)

            try:
                stats = config['fdsn_client'].get_waveforms(
                                                  network=net,
                                                  station=station,
                                                  location='',
                                                  channel='BH*',
                                                  starttime=tbeg-1,
                                                  endtime=tend+1,
                                                  attach_response=True,
                                                 )
            except Exception as E:
                print(E)
                print(" -> get_waveforms() failed...")

            try:
                inv = config['fdsn_client'].get_stations(network=net,
                                                         station=station,
                                                         channel='BH*',
                                                         starttime=tbeg-1,
                                                         endtime=tend+1,
                                                         level='response'
                                                         )
            except Exception as E:
                print(E)
                print(" -> get_stations() failed...")

            if len(stats) > 3:
                print(f" -> merging stream. Length: {len(stats)} -> 3")
                stats.merge(method=1,fill_value="latest")

            stats.sort()
            stats.reverse()
            # stats.remove_response(output="VEL",
            #                       pre_filt=config['prefilt'],
            #                       taper=False,
            #                       zero_mean=True,
            #                      )
            stats.remove_sensitivity()

            #correct mis-alignment
            # stats[0].data, stats[1].data, stats[2].data = rotate2zne(stats[0],0,-90,stats[1],config['subarray_misorientation'][config['subarray_stations'].index(station)],0, stats[2],90+config['subarray_misorientation'][config['subarray_stations'].index(station)],0)
            stats.rotate(method="->ZNE",inventory=inv,components=['Z21'])


            ## new by AB
            stats.resample(sampling_rate=20)
            stats.detrend('linear')
            stats.detrend('simple')

            stats.filter('bandpass',freqmin=config['freq1'],freqmax=config['freq2'],corners=4,zerophase=True) if config['bpf'] == True else None

            stats.trim(tbeg,tend,nearest_sample=False)


            if k == 0:
                o_stats=stats.copy()     #information of the central station
                acc = stats.copy()
                acc.differentiate()

            skip = False
            for n, tr in enumerate(stats):
                if tr.stats.npts != config['samples']:
                    if tr.stats.npts == config['samples']+1:
                        tr.data = tr.data[:-1]
                        print(" -> reducing NPTS by one!") if n == 1 else None
                    else:
                        print(f" -> neglecting {tr.stats.station} due to improper number of samples ({tr.stats.npts} not {config['samples']})") if n == 1 else None
                        skip = True

            if not skip:
                config['subarray'].append(k)
                print(stats)
                try:
#                     if station in ['FFB1','FFB2','FFB3']:
#                         tsz.append(stats.select(station=station, channel="*Z")[0].data)
#                         tsn.append(stats.select(station=station, channel="*1")[0].data)
#                         tse.append(stats.select(station=station, channel="*2")[0].data)
#                     else:
                    tsz.append(stats.select(station=station, channel="*Z")[0].data)
                    tsn.append(stats.select(station=station, channel="*N")[0].data)
                    tse.append(stats.select(station=station, channel="*E")[0].data)
                except:
                    print(" -> stream data could not be appended!")

#    print(np.shape(tsz),np.shape(tsn),np.shape(tse))
    print(f" Array data retrival for {len(config['subarray'])} of {len(config['subarray_mask'])} stations completed!") if config['debug'] else None

    if len(config['subarray']) == 0:
        return np.array(tse), np.array(tsn), np.array(tsz), Stream(), config
    else:
        return np.array(tse), np.array(tsn), np.array(tsz), o_stats, config

In [34]:
def __compute_ADR(tse, tsn, tsz, config, st_template):

    print(' ADR is executing...')

    status_ok = True
    
    try:
        result = AA.array_rotation_strain(np.arange(len(config['subarray'])),
                                          np.transpose(tse),
                                          np.transpose(tsn),
                                          np.transpose(tsz),
                                          config['vp'],
                                          config['vs'],
                                          config['coo'],
                                          config['sigmau'],
                                      )
    except Exception as E:
        print(E)
        print("\n -> failed to compute ADR...")
        status_ok = False 
        return Stream, status_ok
    
    #(Rotation trace)
    rotsa = st_template.copy()
    rotsa[0].data = result['ts_w3']
    rotsa[1].data = result['ts_w2']
    rotsa[2].data = result['ts_w1']
    rotsa[0].stats.channel='BJZ'
    rotsa[1].stats.channel='BJN'
    rotsa[2].stats.channel='BJE'
    rotsa[0].stats.station='RGRF'
    rotsa[1].stats.station='RGRF'
    rotsa[2].stats.station='RGRF'

    rotsa.detrend("simple")

#     gradient_ZNE = result['ts_ptilde'] #u1,1 u1,2 u1,3 u2,1 u2,2 u2,3
#     u_ee=gradient_ZNE[:,0]
#     u_en=gradient_ZNE[:,1]
#     u_ez=gradient_ZNE[:,2]
#     u_ne=gradient_ZNE[:,3]
#     u_nn=gradient_ZNE[:,4]
#     u_nz=gradient_ZNE[:,5]


    #(Gradient trace)
    #      Gradient = o_stats.copy()        #information of the central station
    #      Gradient.append(o_stats[0].copy())
    #      Gradient.append(o_stats[0].copy())
    #      Gradient.append(o_stats[0].copy())
    #      Gradient[0].data = u_ee
    #      Gradient[1].data = u_en
    #      Gradient[2].data = u_ez
    #      Gradient[3].data = u_ne
    #      Gradient[4].data = u_nn
    #      Gradient[5].data = u_nz
    #      Gradient[0].stats.channel='uee'
    #      Gradient[1].stats.channel='uen'
    #      Gradient[2].stats.channel='uez'
    #      Gradient[3].stats.channel='une'
    #      Gradient[4].stats.channel='unn'
    #      Gradient[5].stats.channel='unz'

    return rotsa, status_ok

In [35]:
#### MAIN ##########################


## loop
for date in config['dates']:

    config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray_mask']]

    # print(config['subarray_stations'])
    # print(config['subarray_mask'])

    start_timer1 = timeit.default_timer()
    print(f"\n ____________________________________ \n Working on {date}")

    ## update current date
    config['starttime'] = UTCDateTime(date)

    ## request data
    tse, tsn, tsz, o_stats, config = __get_data(config)

    # print(tse.shape, tsn.shape, tsz.shape)

    ## test if enough stations for ADR are available otherwise continue
    if tse.shape[0] <3 or  tsn.shape[0] <3 or  tsz.shape[0] <3:
        print(" -> not enough stations (< 3) for ADR computation!")
        continue


    ## update stations in subarray
    config['subarray_stations'] = [config['array_stations'][i] for i in config['subarray']]

    ## get inventory and coordinates/distances
    config['coo'] = __get_inventory_and_distances(config)


    ## compute array derived rotation
    rot, status_ok = __compute_ADR(tse, tsn, tsz, config, o_stats)

    if not status_ok:
        print(" -> empty stream for rotations! Skipping...")
        continue
        
    date_str = str(config['starttime'].date).replace("-","")
    config['filename'] = date_str[:6]

    #check if directory is nonexist, then creat
    if os.path.exists(config['save_path']+config['filename']):
        print(' directory %s exists'%(config['save_path']+config['filename']))
    else:
        print(' directory %s is creating...'%(config['save_path']+config['filename']))
        os.mkdir(config['save_path']+config['filename'])
    #    os.touch(config['save_path']+config['filename']+"log.txt")



    print(' data is written ...')
    rot.select(channel="*Z").write(config['save_path']+config['filename']+'/rot_Z_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
    rot.select(channel="*N").write(config['save_path']+config['filename']+'/rot_N_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")
    rot.select(channel="*E").write(config['save_path']+config['filename']+'/rot_E_%s.mseed'%(date_str),format='MSEED', encoding="FLOAT64")

    stop_timer1 = timeit.default_timer()
    print(f"\n Runtime: {round((stop_timer1 - start_timer1)/60,2)} minutes")

stop_timer = timeit.default_timer()
print(f"\n Total runtime: {round((stop_timer - start_timer)/60,2)} minutes")
## End of File


 ____________________________________ 
 Working on 2019-01-01 00:00:00
 requesting GR.FUR..BH*
 requesting BW.FFB1..BH*
 -> reducing NPTS by one!
 -> stream data could not be appended!
 requesting BW.FFB2..BH*
 -> reducing NPTS by one!
 -> stream data could not be appended!
 requesting BW.FFB3..BH*
 -> reducing NPTS by one!
 -> stream data could not be appended!
 requesting BW.GELB..BH*
No data available for request.
Detailed response of server:


 -> get_waveforms() failed...
No data available for request.
Detailed response of server:


 -> get_stations() failed...
 -> stream data could not be appended!
 requesting BW.GRMB..BH*
No data available for request.
Detailed response of server:


 -> get_waveforms() failed...
No data available for request.
Detailed response of server:


 -> get_stations() failed...
 -> stream data could not be appended!
 requesting BW.TON..BH*
 -> merging stream. Length: 5 -> 3
 -> reducing NPTS by one!
 requesting BW.ALFT..BH*
 requesting BW.BIB..BH*
 Array

KeyboardInterrupt: 